In [ ]:
from deep_orderbook.replayer import Replayer
from deep_orderbook.shapper import BookShapper
import numpy as np
from pylab import rcParams

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import clear_output
rcParams['figure.figsize'] = 20, 4

In [ ]:
MARKETS = ["ETHBTC", "BTCUSDT", "ETHUSDT", "BNBBTC", "BNBETH", "BNBUSDT"]

In [ ]:
# itertools for asyncio !
import asyncio
from aioitertools import iter, next, map, zip
from aioitertools import enumerate

In [ ]:
shapper = await BookShapper.create()
file_replayer = Replayer('../crypto-trading/data/L2', date_regexp='2020')
areplay = file_replayer.replayL2_async('ETHBTC', shapper)
b, a, tp, tr = await next(areplay)

In [ ]:
for i in range(100):
    b, a, tp, tr = await next(areplay)
print(f"bids:\n{b.head()}")
print(f"asks:\n{a.head()}")
print(f"prices:\n{tp}")
print(f"trades:\n{tr}")

In [ ]:
replayers = [file_replayer.replayL2_async(pair, await BookShapper.create()) for pair in MARKETS]
multi_replay = file_replayer.multireplayL2_async(replayers)
d = await next(multi_replay)

In [ ]:
async def run(maxpoints=100):
    async for i in iter(range(maxpoints)):
        d = await next(multi_replay)
    #return d

task = asyncio.create_task(run())

In [ ]:
await task

In [ ]:
genarr = shapper.gen_array_async(market_replay=multi_replay, markets=MARKETS)
_ = await next(genarr)

In [ ]:
genacc = shapper.accumulate_array(genarr, markets=MARKETS)
_ = await next(genacc)

In [ ]:
every = 10
LENGTH = 128
x = []
async for n,sec in enumerate(genacc):
    allim = []
    for symb, data in sec.items():
        arr = np.stack(data['bs'][-LENGTH:])
        im = arr
        im[:,:,0] /= 10
        im += 0.5
        allim.append(im)
    allim = np.concatenate(allim, axis=1)
    if n % every == 0:
        plt.figure(figsize=(15,11))
        toshow = allim.transpose(1,0,2)
        toshow = np.clip(toshow, 0, 1)
        plt.imshow(toshow, origin="lower");
        clear_output(wait=True)
        plt.show()
    pass

In [ ]:
genacc = file_replayer.accumulate_array(markets=['ETHBTC'])
next(genacc)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
js = [['0.21654', '0.24654'],['0.21654', '0.24654'],['0.21654', '0.24654'],['0.21654', '0.24654'],['0.21654', '0.24654'],['0.21654', '0.24654']]

In [ ]:
%%timeit
pd.DataFrame(js, columns=['price', 'size']).astype(np.float64).set_index('price')

In [ ]:
%%timeit
[[float(e[0]), float(e[1])] for e in js]